In [1]:
from transformers import BertTokenizerFast, BertForSequenceClassification
from datasets import load_from_disk, Dataset

In [10]:
# S1 提取representation

# S2 finetune raw model

num_labels = 2
max_length = 256

## preprocess

In [3]:
import pandas as pd

pd.set_option("display.max_columns", None)

need_cols = [
    "PRS",
    "sex",
    "height",
    "weight",
    "BSA",
    # "genotype_array",
    "age",
    # "PC1",
    # "PC2",
    # "PC3",
    # "PC4",
    # "PC5",
    # "PC6",
    # "PC7",
    # "PC8",
    # "PC9",
    # "PC10",
    # "assessment_center",
    # "age_squared",
    "BMI",
    "dbp_a",
    "sbp_a",
    "tc_a",
    "tg_a",
    "hdl_a",
    "ldl_a",
    "dbp",
    "sbp",
    "tc",
    "tg",
    "hdl",
    "ldl",
    "apob",
    "cad",
    "mi",
    "crp",
    "eGFR",
    "Cr",
    "CAD",
    "cad_age",
    "death_age",
    # "recuit_age",
    # "year_of_cad_after_recuit",
    # "survival_time",
    # "is_earily_CAD",
    "incident_cad",
    # "cad_status",
    # "prevalent_cad",
    "smoking",
    "prevalent_diabetes",
    "Arrhythmia",
    "Stroke",
    "Peripheral_vascular_disease",
    "Hypertension",
    "future_cad_time_years",
]
train_imputed = pd.read_pickle("result/part1/train_imputed.pkl").set_index("eid")[
    need_cols
]
test_imputed = pd.read_pickle("result/part1/test_imputed.pkl").set_index("eid")[
    need_cols
]


# train_data = pd.read_pickle(
#     "/home/xutingfeng/ukb/project/ppp_prediction/result/dl/ProteomicsBERT/train.pkl"
# )

# test_data = pd.read_pickle(
#     "/home/xutingfeng/ukb/project/ppp_prediction/result/dl/ProteomicsBERT/eval.pkl"
# )
# protein_cols = test_data.columns.tolist()

In [5]:
train_data = pd.read_feather(
    "/home/xutingfeng/ukb/project/ppp_prediction/MulitOmicsDisease/traindata/Prot.feather"
)
protein_cols = train_data.columns.tolist()[1:]

test_data = pd.read_feather(
    "/home/xutingfeng/ukb/project/ppp_prediction/MulitOmicsDisease/traindata/heldout.feather"
)[["eid"] + protein_cols]

train_data.set_index("eid", inplace=True)
test_data.set_index("eid", inplace=True)

train_data

C3      KLK7     GCHFR    NHLRC3     APOD     GAPDH   TP53I3  \
eid                                                                           
1000115  0.040385  0.007345  0.057373 -0.000238  0.02491 -0.065304  0.02669   
1000194 -0.233500  3.085400 -0.536100 -0.009300  0.10330  0.190550 -0.04340   
1000258 -0.192100  0.661400 -0.182000 -0.605800  0.03110 -0.074300  0.40350   
1000533  0.476900  0.115400  0.096500 -0.306100  0.06200 -0.007150 -0.00730   
1000634 -0.184400 -0.289100 -0.016500  0.213800 -0.17840  0.070150  0.12770   
...           ...       ...       ...       ...      ...       ...      ...   
6023470  0.061800 -0.241100 -0.888100  0.371700  0.26640 -0.203400 -1.16820   
6023598 -0.524300 -0.159800 -0.149800  0.071300 -0.04940  0.147200  0.10160   
6023634  0.040385 -0.267400 -0.289100 -0.462900 -0.36910 -0.153150  0.15330   
6023887  0.464600  0.374700 -0.123400  0.289500  0.00580  0.140500  0.80940   
6023931  0.306700  0.007345  0.057373 -0.000238 -0.33020 -0.065304  0.02669   

             CPA4     ANXA2     GRSF1      IL25      HMMR    MRPL52    PAIP2B  \
eid                                                                             
1000115 -0.009212  0.073499  0.082175 -0.019463  0.054938 -0.003103  0.029678   
1000194  0.191500  0.378250  0.666600 -0.543500  0.542750  0.044800 -0.180450   
1000258 -0.836700  0.637800  0.863700  0.391700 -0.008300  0.038500 -0.488450   
1000533  0.157700  0.221850 -0.211200 -0.430200 -0.251350 -0.063300 -0.285450   
1000634 -0.115100  0.000350  0.581100 -0.282300 -0.349850  0.331400  0.068450   
...           ...       ...       ...       ...       ...       ...       ...   
6023470 -0.145200 -0.791800 -0.866900  0.239500 -0.034400 -0.064800 -0.555950   
6023598  0.540200 -0.385900  0.164300  0.254700  0.175400  0.342600 -0.988850   
6023634  0.000000  0.693050  0.504300 -0.032200 -0.171450 -0.582400  0.043650   
6023887 -0.158200  0.738700  0.858800 -0.327300 -0.264000 -0.106800  0.233650   
6023931 -0.009212  0.089700  0.521200  0.075500 -0.028800 -0.577700 -1.058450   

           THAP12       FOS      FGF9    PITHD1     THSD1    PTGES2  \
eid                                                                   
1000115  0.174497 -0.013819 -0.021022  0.161046  0.007754  0.036311   
1000194  1.163600  0.240900  0.857100  2.739650  0.505000 -0.024400   
1000258  0.023400  0.280100  0.367500  0.164700  0.007754  0.036311   
1000533  0.220400 -0.083800  0.687400 -0.181750 -0.096800 -0.027500   
1000634  0.009500  0.205100  0.486900 -0.120450 -0.567900  0.395500   
...           ...       ...       ...       ...       ...       ...   
6023470 -0.363700  0.028700  0.255100  0.233400 -0.185400 -0.667500   
6023598 -0.072100 -0.049600  0.068100 -0.170700 -0.022000  0.228600   
6023634 -0.270300 -0.404100 -0.144700  0.036350 -0.465100 -0.502500   
6023887 -0.317900 -0.189500 -0.090500 -0.022400  0.021000  0.591300   
6023931 -0.443000 -0.142900 -0.628900 -0.796400 -0.138900  0.806100   

         DEFB103A_DEFB103B    ATP1B4     CYB5A     UNC79   SLC34A3    TAGLN3  \
eid                                                                            
1000115           0.046926  0.043311  0.082135  0.058686  0.032686 -0.012728   
1000194          -0.028700  0.140050  0.168400  0.321300 -0.063950 -0.220400   
1000258          -0.173700 -0.015150  0.102100  0.442600  0.127350  0.820600   
1000533           0.038100  0.007850 -0.216100 -0.386600  0.027550 -0.523300   
1000634           0.347700  0.099450  0.143000  0.285100 -0.351250  0.150600   
...                    ...       ...       ...       ...       ...       ...   
6023470          -0.173800  0.257150 -1.186200 -0.307000 -0.719550  0.031700   
6023598          -0.061200  0.273750  0.601300 -0.255300 -0.304450  0.175900   
6023634           0.396900 -0.752650 -0.613700  0.032600  0.366850 -0.167400   
6023887           0.100800  0.284150 -0.346500  0.325100  0.001950 -1.101700   
6023931           0.194900 -0.013350 -0.229100 -0

In [6]:
train_data.index = train_data.index.astype(int)
test_data.index = test_data.index.astype(int)
train_imputed.index = train_imputed.index.astype(int)
test_imputed.index = test_imputed.index.astype(int)

total = pd.concat([train_imputed, test_imputed])

train_data = train_data.merge(total, left_index=True, right_index=True, how="inner")
test_data = test_data.merge(total, left_index=True, right_index=True, how="inner")
test_data

C3      KLK7     GCHFR    NHLRC3     APOD     GAPDH   TP53I3  \
eid                                                                           
1048581  0.547900  0.007345  0.057373 -0.000238 -0.14340 -0.065304  0.02669   
1572870  0.274000  0.476000  0.574200 -0.264900 -0.30550 -0.224700 -0.06780   
5636111  0.815300 -0.269900 -0.311500  0.552200 -0.32930  0.243000  0.31830   
1179667  0.135500  0.251100  0.057373 -0.000238 -0.22730 -0.065304  0.02669   
2359317 -0.259800 -0.499400 -0.503200  0.398700 -0.00120 -0.159500 -0.30510   
...           ...       ...       ...       ...      ...       ...      ...   
4587504 -0.039600  0.109200  0.006400  0.517000 -0.07040  0.800250  0.80820   
5898228 -0.523800  0.007345  0.057373 -0.000238  0.20380 -0.065304  0.02669   
1572853 -0.103900  0.393600  0.019200  0.208300  0.05950  0.041800 -0.17060   
3014647  0.084200 -0.058400  0.182700 -0.120500  0.68100  0.387400 -0.00290   
5898234  0.040385  0.007345  0.057373 -0.000238  0.02491 -0.065304  0.02669   

             CPA4     ANXA2     GRSF1      IL25      HMMR    MRPL52    PAIP2B  \
eid                                                                             
1048581 -0.009212 -0.323000  0.082175 -0.019463  0.054938 -0.003103  0.029678   
1572870  0.047400  0.451300  0.082175 -0.019463  0.054938 -0.003103  0.029678   
5636111 -0.226900 -0.392300 -0.255800 -0.190700  0.007600 -0.233000  0.516750   
1179667 -0.192100  0.471900 -0.268900 -0.377900 -0.690100  0.227900  0.608950   
2359317  0.656600 -0.740400 -0.373800 -0.596200 -0.324600  0.039400 -0.436450   
...           ...       ...       ...       ...       ...       ...       ...   
4587504 -0.683600 -0.073250  0.082175 -0.019463  0.054938 -0.003103  0.029678   
5898228 -0.009212 -0.801000  0.225200 -0.612300  1.307700 -0.355200 -1.300450   
1572853  0.382600 -0.425900  0.082175 -0.019463  0.054938 -0.003103  0.029678   
3014647 -0.358300 -0.125800 -0.161100  0.137600 -0.209000  0.106600 -0.180650   
5898234 -0.009212  0.073499  0.082175 -0.019463  0.054938 -0.003103  0.029678   

           THAP12       FOS      FGF9    PITHD1     THSD1    PTGES2  \
eid                                                                   
1048581  0.174497 -0.013819 -0.021022  0.161046  0.007754  0.036311   
1572870  0.174497 -0.013819 -0.021022  0.161046 -0.883200 -0.027500   
5636111 -0.311400 -0.691400 -0.096700 -0.119900 -0.198700  0.036100   
1179667  0.037600 -0.091800  0.585200 -0.350400 -0.149300  0.024800   
2359317 -0.142400 -0.426200 -0.409000  1.769400 -0.441300  0.278400   
...           ...       ...       ...       ...       ...       ...   
4587504  0.174497 -0.013819 -0.021022  0.161046  0.011300  0.384300   
5898228 -0.303700  0.236500  0.466400 -0.478900 -0.012500 -0.620800   
1572853  0.174497 -0.013819 -0.021022  0.161046  0.033800 -0.253400   
3014647 -0.221100 -0.279500  0.063700  0.145800 -0.404000 -0.178800   
5898234  0.174497 -0.013819 -0.021022  0.161046  0.007754  0.036311   

         DEFB103A_DEFB103B    ATP1B4     CYB5A     UNC79   SLC34A3    TAGLN3  \
eid                                                                            
1048581           0.046926  0.043311  0.082135  0.058686  0.032686 -0.012728   
1572870          -0.234800  0.043311  0.082135  0.058686  0.032686 -0.012728   
5636111           0.122100 -0.303050  0.745100 -0.279000  0.039850  0.650800   
1179667           1.436400 -0.482350  0.392500  0.002800  0.246650 -0.524200   
2359317           0.520400 -0.187350 -0.388900 -0.103500  0.046550 -0.769500   
...                    ...       ...       ...       ...       ...       ...   
4587504          -0.518500  0.043311  0.082135  0.058686  0.032686 -0.012728   
5898228           1.159800  0.523550  2.186300 -0.061200  0.581350 -0.899400   
1572853           0.170100  0.043311  0.082135  0.058686  0.032686 -0.012728   
3014647           1.265800 -0.023950  0.649300 -0.072700 -0.152550 -0.931000   
5898234           0.046926  0.043311  0.082135  0

In [7]:
from datasets import Dataset
from collections import defaultdict
from multiprocessing import cpu_count
import numpy as np
from tqdm.autonotebook import tqdm
import torch


# def castForDF2Dataset(df):
#     dtypes = df.dtypes.to_dict()
#     supported_dtypes = [
#         np.int32,
#         np.float32,
#         np.float64,
#         np.int64,
#         np.float16,
#         np.int16,
#         np.int8,
#         np.uint8,
#         np.uint16,
#         np.uint32,
#         np.uint64,
#         np.bool_,
#     ]
#     for col in df.columns:
#         dtype = df[col].dtypes
#         if dtype in supported_dtypes:
#             continue

#         print(f"{col} has unsupported dtype {dtypes[col]} cast to string")
#         df[col] = df[col].astype(str)

#     return df


def df2dataset(
    df,
    tokenizer,
    max_length=2048,
    features=None,
    attr=None,
    tgt=None,
    tgt_num_classes=2,
    nproc=4,
    batch_size=512,
):
    res = defaultdict(list)

    if features is None and attr is None:
        features = df.columns.tolist()

    ## drop features not in tokenizer

    tokens = list(tokenizer.vocab.keys())
    not_in_list = []
    for col in features:
        if col.lower() not in tokens:  # lower case
            not_in_list.append(col)
    print(
        f"Total {len(not_in_list)} proteins not in tokens will drop them , part of them are : {not_in_list[:10]}"
    )
    print(not_in_list)
    df.drop(columns=not_in_list, inplace=True)
    ### update features
    features = [col for col in features if col in df.columns]

    ## to dict
    for idx, row in tqdm(
        df.iterrows(), total=df.shape[0], desc="processing df to dict"
    ):

        features_row = row[features]
        attr_row = row[attr]

        ranked_row = features_row.sort_values(ascending=False).dropna()

        res["eid"].append(ranked_row.name)
        res["proteins"].append(" ".join(ranked_row.index.tolist()))
        res["values"].append(ranked_row.values.tolist())
        res["length"].append(len(ranked_row))
        res["attr"].append(attr_row.index.tolist())
        res["attr_values"].append(attr_row.values.tolist())
        if tgt is not None:
            # res["label"].append(row[tgt])
            res["labels"].append(
                torch.nn.functional.one_hot(
                    torch.tensor(row[tgt]).long(), num_classes=tgt_num_classes
                ).float()
            )
            res["labels_name"].append(tgt)

    ## to dataset
    dataset = Dataset.from_dict(res)

    ## tokenize data
    def group_texts(examples):

        tokenized_inputs = tokenizer(
            examples["proteins"],
            return_special_tokens_mask=True,
            add_special_tokens=True,
            padding="max_length",
            max_length=max_length,
            truncation=True,
            truncation_strategy="only_first",
        )

        return tokenized_inputs

    dataset = dataset.map(
        group_texts,
        batched=True,
        remove_columns=["proteins"],
        batch_size=batch_size,
        num_proc=min(cpu_count(), 12) if nproc is None else nproc,
    )
    return dataset

In [8]:
tokenizer = BertTokenizerFast.from_pretrained(
    "/home/xutingfeng/ukb/project/ppp_prediction/result/dl/ProteomicsBERT/tokenizer"
)

In [9]:


test_dataset = df2dataset(
    test_data,
    tokenizer,
    features=protein_cols,
    attr=need_cols,
    tgt="incident_cad",
    tgt_num_classes=num_labels,
    max_length=max_length,
)

train_dataset = df2dataset(
    train_data,
    tokenizer,
    features=protein_cols,
    attr=need_cols,
    tgt="incident_cad",
    tgt_num_classes=num_labels,
    max_length=max_length,
)

Total 0 proteins not in tokens will drop them , part of them are : []
[]


processing df to dict:   0%|          | 0/27973 [00:00<?, ?it/s]

Map (num_proc=4):   0%|          | 0/27973 [00:00<?, ? examples/s]

Total 0 proteins not in tokens will drop them , part of them are : []
[]


processing df to dict:   0%|          | 0/23466 [00:00<?, ?it/s]

Map (num_proc=4):   0%|          | 0/23466 [00:00<?, ? examples/s]

In [ ]:
train_dataset.save_to_disk(f"test/train_dataset_{max_length}")
test_dataset.save_to_disk(f"test/test_dataset_{max_length}")

## or load from disk

In [99]:
train_dataset = Dataset.load_from_disk(f"test/train_dataset_{max_length}")
test_dataset = Dataset.load_from_disk(f"test/test_dataset_{max_length}")


Dataset({
    features: ['eid', 'values', 'length', 'attr', 'attr_values', 'labels', 'labels_name', 'input_ids', 'token_type_ids', 'attention_mask', 'special_tokens_mask'],
    num_rows: 24288
})

In [11]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer


model = AutoModelForSequenceClassification.from_pretrained(
    "/home/xutingfeng/ukb/project/ppp_prediction/result/dl/ProteomicsBERT/checkpoint-60000",
    num_labels=num_labels,
)
model

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at /home/xutingfeng/ukb/project/ppp_prediction/result/dl/ProteomicsBERT/checkpoint-60000 and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(2916, 256, padding_idx=0)
      (position_embeddings): Embedding(2916, 256)
      (token_type_embeddings): Embedding(2, 256)
      (LayerNorm): LayerNorm((256,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-5): 6 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=256, out_features=256, bias=True)
              (key): Linear(in_features=256, out_features=256, bias=True)
              (value): Linear(in_features=256, out_features=256, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=256, out_features=256, bias=True)
              (LayerNorm): LayerNorm((256,), eps=1e-12, e

## lightning

In [101]:
import pytorch_lightning as pl

from torch.utils.data import DataLoader

In [102]:
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from torch.utils.data import (
    DataLoader,
    SequentialSampler,
    RandomSampler,
    WeightedRandomSampler,
    Dataset,
)
import torch
import numpy as np


class DatasetModule(pl.LightningDataModule):
    def __init__(
        self,
        train,
        val,
        test,
        batch_size=32,
        num_classes=2,
        y_type="bt",
        num_workers=4,
        weighted=False,
    ):
        super().__init__()

        self.train = train
        self.validation = val
        self.test = test
        self.batch_size = batch_size
        self.num_classes = num_classes
        self.y_type = y_type
        self.weighted = weighted
        self.num_workers = num_workers
        self._init_dataset()

    def _init_dataset(self):
        train, val, test = self.train, self.validation, self.test

        if self.y_type == "bt":

            class_weights = dict(
                enumerate(
                    class_weight.compute_class_weight(
                        "balanced",
                        classes=np.arange(self.num_classes),
                        y=torch.argmax(train["labels"], dim=1).numpy(),
                    )
                )
            )
            self.class_weights = class_weights

    def train_dataloader(self):

        if self.y_type == "bt" and self.weighted:
            print("Using weighted sampler")
            train_class_weights = [
                self.class_weights[torch.argmax(i).item()] for i in self.train["labels"]
            ]
            sampler = WeightedRandomSampler(
                train_class_weights, len(train_class_weights), replacement=True
            )
        else:
            sampler = RandomSampler(self.train)

        return DataLoader(
            self.train,
            batch_size=self.batch_size,
            sampler=sampler,
            drop_last=True,
            persistent_workers=True,
            num_workers=self.num_workers,
        )

    def val_dataloader(self):
        return DataLoader(
            self.validation,
            batch_size=self.batch_size,
            persistent_workers=True,
            num_workers=self.num_workers,
            sampler=SequentialSampler(self.validation),
        )

    def test_dataloader(self):
        return DataLoader(
            self.test,
            batch_size=self.batch_size,
            persistent_workers=True,
            num_workers=self.num_workers,
            sampler=SequentialSampler(self.test),
        )

In [236]:
from torch import nn
import torch.nn.functional as F
import torchmetrics
from collections import defaultdict

HF_INPUT_KEYS = ["input_ids", "attention_mask", "token_type_ids", "labels"]


class PlasmaLLM_pl(pl.LightningModule):
    def __init__(
        self,
        model,
        lr=1e-3,
        weight_decay=1e-2,
        loss=None,
        **kwargs,
    ):

        super(PlasmaLLM_pl, self).__init__()
        self.model = model
        self.lr = lr
        self.weight_decay = weight_decay

        self.mertic = {
            "train_auc": torchmetrics.AUROC(num_classes=2, task="multiclass"),
            "val_auc": torchmetrics.AUROC(num_classes=2, task="multiclass"),
        }
        self.history = defaultdict(dict)
        # self.loss_fn = nn.CrossEntropyLoss(weight=torch.tensor(weight).float())
        self.loss = loss

    def forward(self, x):
        if isinstance(x, (list, tuple)):
            return self.model(*x)
        elif isinstance(x, dict):
            passed_dict = {}
            for k, v in x.items():
                if k in HF_INPUT_KEYS:
                    # passed_dict[k] = v
                    passed_dict[k] = v.to(self.device)
                    # if k in ["labels", "input_ids"]:
                    #     passed_dict[k] = v.to(self.device)
                    # else:
                    #     passed_dict[k] = v

            return self.model(**passed_dict)

    def training_step(self, train_batch, batch_idx):
        if isinstance(train_batch, dict):
            if self.loss is not None:
                x = {k: v for k, v in train_batch.items() if k in HF_INPUT_KEYS}
                x.pop("labels")

                y = train_batch["labels"]
                o = self.forward(x)
                outputs = o[0]  # c output
                loss = self.loss(outputs, y.squeeze(-1))
            else:
                x = {k: v for k, v in train_batch.items() if k in HF_INPUT_KEYS}
                y = train_batch["labels"]
                o = self.forward(x)
                loss = o[0]
                outputs = o[1]

        # else:
        #     x, y = train_batch
        #     outputs = self.forward(x)
        #     loss = self.loss_fn(outputs, y.squeeze(-1).float())

        self.mertic["train_auc"].update(
            torch.softmax(outputs, dim=-1), torch.argmax(y, dim=1)
        )

        self.log("ptl/train_loss", loss, on_epoch=True, prog_bar=True, on_step=False)
        return loss

    def validation_step(self, val_batch, batch_idx):
        if isinstance(val_batch, dict):
            if self.loss is not None:
                x = {k: v for k, v in val_batch.items() if k in HF_INPUT_KEYS}
                x.pop("labels")
                o = self.forward(x)
                y = val_batch["labels"]

                outputs = o[0]  # c output
                loss = self.loss(outputs, y.squeeze(-1))
            else:
                x = {k: v for k, v in val_batch.items() if k in HF_INPUT_KEYS}
                y = val_batch["labels"]
                o = self.forward(x)
                loss = o[0]
                outputs = o[1]
            # print(outputs)

        # else:
        #     x, y = val_batch
        #     outputs = self.forward(x).logits
        #     loss = self.loss_fn(outputs, y.squeeze(-1).float())

        self.mertic["val_auc"].update(
            torch.softmax(outputs, dim=-1), torch.argmax(y, dim=1)
        )

        self.log("ptl/val_loss", loss, on_epoch=True, prog_bar=True)

    def on_train_epoch_end(self):

        auc = self.mertic["train_auc"].compute()
        self.log("ptl/train_auc", auc, prog_bar=True)

    def on_validation_epoch_end(self):
        auc = self.mertic["val_auc"].compute()
        self.log("ptl/val_auc", auc, prog_bar=True)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(
            self.parameters(), lr=self.lr, weight_decay=self.weight_decay
        )
        return optimizer

    def predict_dataset(self, dataset, batch_size=256):
        self.eval()
        dataloader = DataLoader(
            dataset,
            batch_size=batch_size,
            persistent_workers=True,
            num_workers=4,
            shuffle=False,
        )

        pred = []
        with torch.no_grad():
            for x in dataloader:
                o = self.forward(x)
                y_hat = torch.softmax(o[1], dim=-1)[:, 1]
                pred.append(y_hat)
        pred = torch.cat(pred).cpu().detach().numpy()
        return pred

    def predict_df(self, df, batch_size=256):

        for feature in self.features:
            assert feature in df.columns
        print(f"input df have NA: {df[self.features].isna().sum(axis=1).sum()}")
        df = df.copy().dropna(subset=self.features)

        predict_dataloader = DataLoader(
            torch.tensor(df[self.features].values).float(),
            batch_size=batch_size,
            persistent_workers=True,
            num_workers=4,
        )

        self.eval()
        pred = []
        with torch.no_grad():
            for x in predict_dataloader:
                y_hat = self.forward(x).cpu().detach()
                y_hat = torch.softmax(y_hat, dim=-1)[:, 1]

                pred.append(y_hat)
        pred = torch.cat(pred).cpu().detach().numpy()
        df["pred"] = pred
        return df

In [229]:
from datasets import Dataset

train_dataset = Dataset.load_from_disk("test/train_dataset1")
test_dataset = Dataset.load_from_disk("test/test_dataset1")


total = train_dataset.train_test_split(test_size=0.2)
train_dataset = total["train"]
val_dataset = total["test"]
del total
train_dataset = train_dataset.with_format("torch")
val_dataset = val_dataset.with_format("torch")
test_dataset = test_dataset.with_format("torch")

dm = DatasetModule(
    train_dataset,
    val_dataset,
    test_dataset,
    batch_size=128,
    num_classes=2,
    weighted=False,
)

In [202]:
dm.class_weights

{0: 0.5294566461387542, 1: 8.98704902867715}

In [203]:
def frozen_bert(
    model,
    freeze_layers=0,
    tune_embeding_layer=False,
    tune_attn_only=False,
):
    if tune_embeding_layer:
        print("Tuning embedding layer")
    else:
        print("Freezing embedding layer")
        for param in model.bert.embeddings.parameters():
            param.requires_grad = False

    if freeze_layers > 0:
        modules_to_freeze = model.bert.encoder.layer[:freeze_layers]
        for module in modules_to_freeze:
            for param in module.parameters():
                param.requires_grad = False
    if len(model.bert.encoder.layer[freeze_layers:]) > 0:
        if not tune_attn_only:
            print("Tuning all layers")
        else:
            print("Tuning only attn")
            for module in model.bert.encoder.layer[freeze_layers:]:
                for param in module.intermediate.parameters():
                    param.requires_grad = False
                for param in module.output.parameters():
                    param.requires_grad = False

    print("***********************trainable layers ***********************")
    for name, param in model.named_parameters():
        if param.requires_grad:
            print(name)
    print("***********************Fixed layers ***********************")
    for name, param in model.named_parameters():
        if not param.requires_grad:
            print(name)
    print("***********************End ***********************")

In [204]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

num_labels = 2
model = AutoModelForSequenceClassification.from_pretrained(
    "/home/xutingfeng/ukb/project/ppp_prediction/result/dl/ProteomicsBERT/checkpoint-60000",
    num_labels=num_labels,
)
model

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at /home/xutingfeng/ukb/project/ppp_prediction/result/dl/ProteomicsBERT/checkpoint-60000 and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(2916, 256, padding_idx=0)
      (position_embeddings): Embedding(2916, 256)
      (token_type_embeddings): Embedding(2, 256)
      (LayerNorm): LayerNorm((256,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-5): 6 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=256, out_features=256, bias=True)
              (key): Linear(in_features=256, out_features=256, bias=True)
              (value): Linear(in_features=256, out_features=256, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=256, out_features=256, bias=True)
              (LayerNorm): LayerNorm((256,), eps=1e-12, e

In [205]:
frozen_bert(model, freeze_layers=3, tune_embeding_layer=True, tune_attn_only=False)

Tuning embedding layer
Tuning all layers
***********************trainable layers ***********************
bert.embeddings.word_embeddings.weight
bert.embeddings.position_embeddings.weight
bert.embeddings.token_type_embeddings.weight
bert.embeddings.LayerNorm.weight
bert.embeddings.LayerNorm.bias
bert.encoder.layer.3.attention.self.query.weight
bert.encoder.layer.3.attention.self.query.bias
bert.encoder.layer.3.attention.self.key.weight
bert.encoder.layer.3.attention.self.key.bias
bert.encoder.layer.3.attention.self.value.weight
bert.encoder.layer.3.attention.self.value.bias
bert.encoder.layer.3.attention.output.dense.weight
bert.encoder.layer.3.attention.output.dense.bias
bert.encoder.layer.3.attention.output.LayerNorm.weight
bert.encoder.layer.3.attention.output.LayerNorm.bias
bert.encoder.layer.3.intermediate.dense.weight
bert.encoder.layer.3.intermediate.dense.bias
bert.encoder.layer.3.output.dense.weight
bert.encoder.layer.3.output.dense.bias
bert.encoder.layer.3.output.LayerNorm.we

In [206]:
model_pl = PlasmaLLM_pl(
    model,
    lr=1e-4,
    weight_decay=1e-4,
    loss=nn.CrossEntropyLoss(weight=torch.tensor([1, 5]).float()),
)

In [207]:
from pytorch_lightning import Trainer

trainer = Trainer(
    max_epochs=10,
    gradient_clip_val=1,
)
trainer.fit(model_pl, dm)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type                          | Params
--------------------------------------------------------
0 | model | BertForSequenceClassification | 4.7 M 
1 | loss  | CrossEntropyLoss              | 0     
--------------------------------------------------------
3.1 M     Trainable params
1.6 M     Non-trainable params
4.7 M     Total params
18.892    Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.


In [237]:
model_pl.to("cuda")
test_dataset.set_format("torch")
o = model_pl.predict_dataset(
    test_dataset,
)
test_dataset.set_format("pandas")
test_df = pd.DataFrame(
    {
        "eid": test_dataset["eid"],
        "labels": test_dataset["labels"].apply(lambda x: x[1]),
        "pred": o,
    }
)

from ppp_prediction.corr import cal_binary_metrics

cal_binary_metrics(test_df["labels"].values, test_df["pred"].values)

,eid,labels,pred
0,1982486,0.0,0.011926
1,5937359,0.0,0.079876
2,3737374,0.0,0.003899
3,5525566,0.0,0.005558
4,1828132,0.0,0.002873
...,...,...,...
3995,2200051,0.0,0.003659
3996,4095573,0.0,0.003418
3997,5600706,0.0,0.005188
3998,5763542,1.0,0.019781


In [221]:
for i in dm.test_dataloader():
    print(i)
    break

import pandas as pd

{'eid': tensor([1982486, 5937359, 3737374, 5525566, 1828132, 1498921, 1903607, 2352592,
        3474825, 2844166, 5348387, 5431137, 2566858, 2425703, 5920083, 4997808,
        2278006, 2386342, 3449929, 5304865, 2670888, 4912963, 4120672, 1208691,
        1523440, 5903051, 2339819, 4014974, 4055945, 2507467, 1206817, 3954840,
        3020292, 3541208, 4560666, 2871162, 1864681, 2771870, 5155296, 4545042,
        4351005, 4575027, 1588273, 4692433, 2369926, 3708006, 4774498, 1965015,
        4203972, 3499901, 1614841, 1578991, 1022613, 1059797, 1944491, 2301716,
        3486707, 1799561, 4682430, 1312044, 2767973, 3053105, 5269087, 2265661,
        1344263, 3619211, 5519769, 5910124, 5536735, 3488210, 4932595, 2923469,
        4451477, 2069329, 4077271, 3502625, 1694986, 4190226, 3778549, 1775141,
        4542701, 1004027, 5716308, 3826958, 4226898, 2472028, 5951259, 3648325,
        3143633, 4635512, 1974947, 4403565, 2152760, 1639493, 1637552, 2785394,
        1029438, 4913207, 276945

In [223]:
model_pl.eval()
model_pl.to("cpu")
o = model_pl.forward(i)
pd.DataFrame(
    {"true": i["labels"][:, 1], "pred": torch.argmax(o[1], dim=1)}
).sort_values("pred")

,true,pred
0,0.0,0
92,0.0,0
91,0.0,0
90,0.0,0
89,0.0,0
...,...,...
35,0.0,0
79,0.0,1
25,0.0,1
29,0.0,1


In [209]:
model_pl.eval()
model_pl.to("cpu")
i.pop("labels")
o = model_pl.forward(i)

In [210]:
len(o)

1

In [211]:
model_pl.predict_df

<bound method PlasmaLLM_pl.predict_df of PlasmaLLM_pl(
  (model): BertForSequenceClassification(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(2916, 256, padding_idx=0)
        (position_embeddings): Embedding(2916, 256)
        (token_type_embeddings): Embedding(2, 256)
        (LayerNorm): LayerNorm((256,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0-5): 6 x BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=256, out_features=256, bias=True)
                (key): Linear(in_features=256, out_features=256, bias=True)
                (value): Linear(in_features=256, out_features=256, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Line

In [213]:
torch.argmax(o[0], dim=1)

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])

In [214]:
import pandas as pd

pd.DataFrame({"true": i["labels"][:, 1], "pred": torch.argmax(o[1], dim=1)})

KeyError: 'labels'

In [ ]:
keys = ["input_ids", "attention_mask", "label"]
model(
    **{
        "input_ids": i["input_ids"],
        "attention_mask": i["attention_mask"],
        "labels": i["label"],
    }
)

In [ ]:
dataloader = DataLoader(train_dataset.with_format("torch"), batch_size=8, shuffle=True)
dataloader

In [ ]:
for batch in dataloader:
    print(batch)
    break

## hf

In [ ]:
from datasets import load_dataset, load_metric

# metric = load_metric("roc_auc")
metric = load_metric("accuracy")

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred

    predictions = np.argmax(predictions, axis=1)
    labels = np.argmax(labels, axis=1)
    # print(predictions, labels)
    res = metric.compute(predictions=predictions, references=labels)

    return res
    # return metric.compute(predictions=predictions, references=labels)

In [ ]:
# frozen_model
class CONFIG:
    def __init__(self, **kwargs):
        for k, v in kwargs.items():
            setattr(self, k, v)


config = CONFIG(
    freeze_layers=5,
    tune_embeding_layer=False,
    tune_attn_only=False,
)


# config = vars(config)

####### Freeze layers #####

if config.freeze_layers is not None:
    def_freeze_layers = config.freeze_layers

if config.tune_embeding_layer:
    print("Tuning embedding layer")

else:
    print("Freezing embedding layer")
    for param in model.bert.embeddings.parameters():
        param.requires_grad = False

if def_freeze_layers > 0:
    modules_to_freeze = model.bert.encoder.layer[:def_freeze_layers]
    for module in modules_to_freeze:
        for param in module.parameters():
            param.requires_grad = False

if len(model.bert.encoder.layer[def_freeze_layers:]) > 0:
    if not config.tune_attn_only:
        print("Tuning all layers")
    else:
        print("Tuning only attn")
        for module in model.bert.encoder.layer[def_freeze_layers:]:

            for param in module.intermediate.parameters():
                param.requires_grad = False
            for param in module.output.parameters():
                param.requires_grad = False

print("***********************trainable layers ***********************")
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name)
print("***********************Fixed layers ***********************")
for name, param in model.named_parameters():
    if not param.requires_grad:
        print(name)
print("***********************End ***********************")

In [ ]:
training_args = TrainingArguments(
    output_dir="test",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    per_device_train_batch_size=128,
    per_device_eval_batch_size=128,
    weight_decay=0.01,
    push_to_hub=False,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

In [ ]:
for batch in trainer.get_eval_dataloader():
    break

In [ ]:
model.eval()
outputs = model(**batch)
outputs

In [ ]:
outputs

In [ ]:
batch["labels"]

In [ ]:
from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained(
    "/home/xutingfeng/ukb/project/ppp_prediction/result/dl/ProteomicsBERT/checkpoint-60000"
)
model

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="test",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    weight_decay=0.01,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=test_dataset,
    eval_dataset=test_dataset,
    data_collator=data_collator,
)

# trainer.train()

In [ ]:
for batch in trainer.get_train_dataloader():
    break

In [ ]:
batch["labels"]

In [ ]:
model.eval()
o = model(**batch)
o

In [ ]:
batch["labels"][:, :20]

In [ ]:
import torch

torch.argmax(o["logits"], dim=2)[:, :20]

In [ ]:
batch["input_ids"][:, :20]

In [ ]:
tokenizer.decode(batch["input_ids"][0, :20])

In [ ]:
tokenizer.decode(torch.argmax(o["logits"], dim=2)[0, :20])

In [ ]:
o

In [ ]:
from datasets import load_from_disk

test_data = load_from_disk(
    "/home/xutingfeng/ukb/project/ppp_prediction/result/dl/ProteomicsBERT/test"
)
test_data

In [ ]:
test_data = test_data.select_columns(
    ["input_ids", "token_type_ids", "attention_mask", "special_tokens_mask"]
)
test_data

In [ ]:
model.eval()

In [ ]:
torch.argmax(o["logits"][0, :3], dim=1)

In [ ]:
o["logits"][0, :3].shape